In [1]:
print("ok")

ok


In [2]:
# --- ADD THIS CELL TO YOUR NOTEBOOK ---
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

print("🚀 Creating FAISS index...")

# Load and process the PDF
file_path = "../Data/Medical_book.pdf"  # Note the ../ to go up one level
loader = PyPDFLoader(file_path)
docs = loader.load()
print(f"📖 Loaded {len(docs)} pages")

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"✂️ Created {len(chunks)} text chunks")

# Create embeddings and FAISS index
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save the index (saves to main project folder)
vectorstore.save_local("../faiss_index")
print("✅ FAISS index created at '../faiss_index'")

🚀 Creating FAISS index...
📖 Loaded 637 pages
✂️ Created 6600 text chunks


/var/folders/nl/q3f58lb91kl7sqnyg6y9r4dr0000gn/T/ipykernel_31102/3351946085.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/apple/miniconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ FAISS index created at '../faiss_index'


In [3]:
%pwd

'/Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI/research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'/Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [12]:
from langchain.document_loaders import PyPDFLoader

# Replace with the actual path to your PDF file
loader = PyPDFLoader("/Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI/Data/Medical_book.pdf")  # Update this with your file's location
extracted_data = loader.load()


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
try:
    from langchain_huggingface import HuggingFaceEmbeddings # type: ignore
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings  # fallback

In [16]:
try:
    from langchain_huggingface import HuggingFaceEmbeddings # type: ignore
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

# Check if index already exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index '{index_name}' already exists.")


Index 'medicalbot' already exists.


In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



In [22]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [23]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch


In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[Document(id='9b5d3509-6416-4954-bba7-bf288572503d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI/Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c6f9c47f-7d40-4eda-b85a-35f21f405895', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='6a2de801-bed3-4cf4-8067-b7e1099bc9c4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'Py

In [28]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4, max_tokens=500)


In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# System prompt and prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say so. Be concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Make sure you are using the 3.5 model here:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4, max_tokens=500)

# Create chain properly using the same LLM
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assuming 'retriever' is already set
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [33]:
docs = retriever.invoke("what is Acne?")
print(docs[0].page_content if docs else "No docs found.")


GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26


In [34]:
print(len(docs))  # Number of documents retrieved
print(sum(len(d.page_content.split()) for d in docs))  # Approx total words


3
54


In [35]:
import logging
logging.basicConfig(level=logging.INFO)


In [36]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.4, max_tokens=1000)


In [37]:
# Path to your PDF
file_path = "../Data/Medical_book.pdf"
data_dir = "../Data"

# Debug: Print current working directory and list files in the actual data directory
print("Current working directory:", os.getcwd())
print("Looking for 'Data' directory at:", os.path.abspath(data_dir))
if os.path.exists(data_dir):
    print("Files in '../Data':", os.listdir(data_dir))
else:
    print("'Data' directory does not exist at", os.path.abspath(data_dir))

# Check if file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}. Please check the path and ensure the file exists.")
else:
    # Load PDF
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    # Split into chunks for better retrieval
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(docs)

    # Generate embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create FAISS vectorstore
    vectorstore = FAISS.from_documents(chunks, embeddings)
    print(f"FAISS vectorstore created with {len(chunks)} chunks.")


Current working directory: /Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI
Looking for 'Data' directory at: /Users/apple/Desktop/Data
'Data' directory does not exist at /Users/apple/Desktop/Data
File not found: ../Data/Medical_book.pdf. Please check the path and ensure the file exists.


In [38]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI



In [39]:
# Use the existing retriever and llm
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [40]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":1})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)


In [41]:
print("API Key Set:", "OPENAI_API_KEY" in os.environ)
print("Vectorstore Exists:", 'vectorstore' in globals())
print("QA Chain Exists:", 'qa_chain' in globals())


API Key Set: True
Vectorstore Exists: True
QA Chain Exists: True


In [42]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

file_path = "Data/Medical_book.pdf"
data_dir = "Data"

print("Current working directory:", os.getcwd())
print("Looking for 'Data' directory at:", os.path.abspath(data_dir))
if os.path.exists(data_dir):
    print("Files in 'Data':", os.listdir(data_dir))
else:
    print("'Data' directory does not exist at", os.path.abspath(data_dir))

if not os.path.exists(file_path):
    print(f"File not found: {file_path}. Please check the path and ensure the file exists.")
else:
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    vectorstore = FAISS.from_documents(chunks, embeddings)
    print(f"✅ FAISS vectorstore created with {len(chunks)} chunks.")


Current working directory: /Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI
Looking for 'Data' directory at: /Users/apple/Desktop/End-to-end-Medical-Chatbot-Generative-AI/Data
Files in 'Data': ['Medical_book.pdf']


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅ FAISS vectorstore created with 6600 chunks.


In [43]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":1})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

print("✅ QA Chain is ready!")


✅ QA Chain is ready!


In [44]:
%pip install hf_xet


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [45]:
%pip install transformers torch accelerate bitsandbytes

from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
import os

# Fix tokenizer warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("🚀 Loading free medical model...")

# Use a small, fast model that's good for medical Q&A
model_name = "microsoft/DialoGPT-medium"  # Free, fast, good for conversations

try:
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    llm = HuggingFacePipeline(pipeline=pipe)
    
    # Custom prompt for medical Q&A
    prompt_template = """Use the following medical context to answer the question accurately and helpfully.

Medical Context:
{context}

Question: {question}

Provide a clear, concise medical answer based only on the context above. If the context doesn't contain the answer, say "I don't have enough medical information about this."
"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    
    # Update your QA chain with better prompt
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )
    
    print("✅ Free local model loaded successfully!")
    
    # Test it immediately
    query = "What is Acne?"
    print(f"🧪 Testing with: '{query}'")
    
    response = qa_chain.invoke({"query": query})
    print("\n" + "="*60)
    print("🤖 MEDICAL BOT RESPONSE:")
    print("="*60)
    print(response['result'])
    print("="*60)
    
    # Show source information
    print(f"\n📚 Retrieved {len(response['source_documents'])} relevant medical documents")
    for i, doc in enumerate(response['source_documents'][:2]):
        print(f"\nSource {i+1}:")
        print(f"Page: {doc.metadata.get('page', 'N/A')}")
        print(f"Content: {doc.page_content[:150]}...")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Trying an even simpler approach...")
    
    # Fallback: Direct retrieval + generation
    def ask_medical_question(question):
        docs = retriever.invoke(question)
        context = "\n".join([doc.page_content for doc in docs[:3]])
        
        prompt = f"""Based on this medical information:

{context}

Question: {question}

Please provide a clear answer:"""
        
        response = llm.invoke(prompt)
        return response, docs
    
    # Test fallback
    query = "What is Acne?"
    answer, sources = ask_medical_question(query)
    print("\n🤖 Medical Answer (Fallback):")
    print(answer)
    print(f"\n📚 Retrieved {len(sources)} relevant documents")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
🚀 Loading free medical model...


Device set to use mps:0
/var/folders/nl/q3f58lb91kl7sqnyg6y9r4dr0000gn/T/ipykernel_31102/782170015.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


✅ Free local model loaded successfully!
🧪 Testing with: 'What is Acne?'

🤖 MEDICAL BOT RESPONSE:
Use the following medical context to answer the question accurately and helpfully.

Medical Context:
Journal of Urology(Mar. 1998): 935-940.
Nancy J. Nordenson
Acid reflux see Heartburn
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 million

Question: What is Acne?

Provide a clear, concise medical answer based only on the context above. If the context doesn't contain the answer, say "I don't have enough medical information about this."


📚 Retrieved 1 relevant medical documents

Source 1:
Page: 37
Content: Journal of Urology(Mar. 1998): 935-940.
Nancy J. N

In [46]:
%pip install streamlit

# Create a new file: medical_chatbot_app.py

Note: you may need to restart the kernel to use updated packages.


In [47]:
# Save the FAISS index for the Streamlit app
vectorstore.save_local("faiss_index")
print("✅ FAISS index saved as 'faiss_index' folder")

✅ FAISS index saved as 'faiss_index' folder


In [48]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Check if FAISS index exists
if os.path.exists("faiss_index"):
    print("✅ FAISS index folder exists")
    files = os.listdir("faiss_index")
    print(f"📁 Files in faiss_index: {files}")
else:
    print("❌ FAISS index folder not found")
    print("Please run the notebook to create the FAISS index first")

✅ FAISS index folder exists
📁 Files in faiss_index: ['index.faiss', 'index.pkl']
